In [87]:
# %conda info
# !pip install -qU openai pinecone-client datasets
#!pip install markdown
#!pip install --upgrade openai 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 kB 1.6 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 0.26.5
    Uninstalling openai-0.26.5:
      Successfully uninstalled openai-0.26.5


In [5]:
import openai

# get API key from top-right dropdown on OpenAI website
openai.api_key = "***"
embed_model = "text-embedding-ada-002"

In [17]:
def complete(prompt):
    # query text-davinci-003
    res = openai.Completion.create(
        engine='gpt-3.5-turbo', #'text-davinci-003',
        prompt=prompt,
        temperature=0,
        max_tokens=400,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )
    return res['choices'][0]['text'].strip()

def complete_gpt3_5(prompt):
    res = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
            {"role": "system", "content": "You are a helpful and clairvoyant savant and oracle who tries to best answer the users query using just the context they provide."},
            {"role": "user", "content": prompt}
        ]
    )
    return res['choices'][0]['message']['content']

def complete_gpt_4(prompt):
    res = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[
            {"role": "system", "content": "You are a helpful assistant that elaborates on the users query using only the context they provide. If the context does not provide sufficient details for you to formulate an answer you politely let them know."},
            {"role": "user", "content": prompt}
        ]
    )
    return res['choices'][0]['message']['content']


query = "Who was the first man to walk on the moon?"
import time
start = time.time()
complete_gpt3_5(query)
end = time.time()
print(start - end)

-1.8579602241516113


In [42]:
import re, os, hashlib, fnmatch

def search_image_files(filename, directory):
    for dirpath, dirnames, filenames in os.walk(directory):
        for f in filenames:
            if f == filename:
                ext = os.path.splitext(f)[1].lower()
                if ext in ('.png', '.jpg', '.jpeg', '.gif'):
                    return os.path.join(dirpath, f)
    return None

def get_all_attachments_in_text(note_string):
    regex_pattern = r"\[\[.*?\]\]|!\[\[.*?\]\]"
    strip_char = r"\[|\]|!"
    matches = [re.sub(strip_char,'',match) for match in re.findall(regex_pattern,  note_string) ]
    return matches

def remove_urls(text):
    # Regular expression pattern for matching URLs
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    # Remove URLs from the text
    without_urls = re.sub(url_pattern, '', text)
    return without_urls

def remove_obsidian_links(text):
    clean = re.compile('\[\[.*?\]\]|!\[\[.*?\]\]')
    return re.sub(clean, '', text)

def parse_markdown_file(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()
    _, filename = os.path.split(file_path)
    # Initialize a counter to keep track of how many '---'s we've seen
    count = 0
    for i, line in enumerate(lines):
        # If we've seen one '---' already and we've just seen another, return the rest of the lines
        if line == '---\n' and count == 1:
            count += 1
            lines = lines[i:]
            break
        # If we've just seen our first 'b', increment the count
        elif line == '---\n' and count == 0:
            count += 1

    print(count)
    # insert the filename as first element
    lines.insert(0, filename + "\n")
    return lines


def get_markdown_files(directory):
    markdown_files = []
    for item in os.listdir(directory):
        if item == ".obsidian" or item == "Daily":
            continue
        item_path = os.path.join(directory, item)
        if os.path.isdir(item_path):
            markdown_files.extend(get_markdown_files(item_path))
        elif item.endswith('.md'):
            markdown_files.append(item_path)
    return markdown_files

def split_list(note_lines_list, word_threshold):
    sublists = []
    sublist = []
    subtotal = 0
    for sentence in note_lines_list:
        sublist.append(sentence)
        word_count = len(sentence.split())
        if subtotal + word_count > word_threshold:
            sublists.append(sublist)
            sublist = []
            subtotal = 0
        subtotal += word_count
    if sublist:
        sublists.append(sublist)
    return sublists


def get_files_to_index(rootdir, searchstr='publish: true'):
    files_to_index = []
    for subdir, dirs, files in os.walk(rootdir):
        for file in files:
            if fnmatch.fnmatch(file, '*.md') == True:
                filepath = os.path.join(subdir, file)
                if fnmatch.fnmatch(filepath,'*/.trash/*') == False:
                    with open(filepath, 'r') as f:
                        if searchstr in f.read():
                            files_to_index.append(filepath)
    return files_to_index



In [43]:
#parse_markdown_file("/Users/ammarh/Documents/second-brain/command-center.md")
parse_markdown_file("/Users/ammarh/Documents/second-brain/3 - Resources/ai-ml/reinforcement learning.md")

2


['reinforcement learning.md\n',
 '---\n',
 '\n',
 '\n',
 '---\n',
 '\n',
 'Courses :\n',
 '[[xcs234 - reinforcement learning]]\n',
 'xcs229ii - [[3 - reinforcement learning]]\n',
 '[[huggingface - deeprl]]\n',
 '\n',
 'Random tinkering:\n',
 '[Reinforcement Learning Tips and Tricks — Stable Baselines3 1.8.0a3 documentation](https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html)\n',
 '[Part 1: Key Concepts in RL — Spinning Up documentation](https://spinningup.openai.com/en/latest/spinningup/rl_intro.html)\n',
 '[Train a Mario-playing RL Agent — PyTorch Tutorials 1.12.1+cu102 documentation](https://pytorch.org/tutorials/intermediate/mario_rl_tutorial.html) : [Colab with tinkered code](https://colab.research.google.com/drive/1DtWyTd0P-ZPXIojn3RhxIRf9HszTQnmz?authuser=1#scrollTo=skXH2HhsW_Gn)\n',
 '\n',
 '- [x] #goal #_2023 Do the huggingface DeepRL course [[huggingface - deeprl]] ✅ 2023-02-07\n']

In [38]:
fo = [1,2,3,4,5,6,7]
fo = fo[2:]
print(fo)
fo.insert(0,0)
print(fo)



[3, 4, 5, 6, 7]
[0, 3, 4, 5, 6, 7]


In [78]:
files_list = get_files_to_index("/Users/ammarh/Documents/second-brain/")
len(files_list)

582

In [68]:
files_list = get_files_to_index("/Users/ammarh/Documents/second-brain/")
# files_list = get_markdown_files("/Users/ammarh/Documents/test-vault/.")

VECTOR_WORD_LIMIT = 800

notes_data = []

for file in files_list:
    note_lines = parse_markdown_file(file)        
    note_text_split = split_list(note_lines, VECTOR_WORD_LIMIT)
    for i,note_text_split_snippet in enumerate(note_text_split):
        note_snippet = "".join(note_text_split_snippet)
        notes_data.append({
            'uuid': hashlib.sha256((file + "_^_" + str(i)).encode()).hexdigest(),
            'file': file,
            'section': i,
            'note': note_snippet
        })

print(f"Adding {len(notes_data)} chunks from {len(files_list)} files")


Adding 329 chunks from 256 files


In [69]:
files = set([n['file'] for n in notes_data])
a = [f for f in files if "learning" in f]
print(a)

['/Users/ammarh/Documents/second-brain/2 - Areas/machine-learning/machine-learning.md', '/Users/ammarh/Documents/second-brain/3 - Resources/courses/xcs234 - reinforcement learning/xcs234 - reinforcement learning.md', '/Users/ammarh/Documents/second-brain/2 - Areas/machine-learning/shiny new models.md']


In [70]:

import pinecone

index_name = 'obsidian-second-brain'

# initialize connection to pinecone (get API key at app.pinecone.io)
pinecone.init(
    api_key="***",
    environment="us-east1-gcp"
)

# check if index already exists (it shouldn't if this is first time)
if index_name not in pinecone.list_indexes():
    print("creating new index")
    # if does not exist, create index
    pinecone.create_index(
        index_name,
        dimension=1536,
        metric='cosine',
        metadata_config={'indexed': ['file']}
    )
# connect to index
index = pinecone.Index(index_name)
# view index stats
# index.describe_index_stats()

creating new index


In [71]:
from tqdm.auto import tqdm
from time import sleep

batch_size = 100  # how many embeddings we create and insert at once

for i in tqdm(range(0, len(notes_data), batch_size)):
    #print(f"i - {i}")
    # find end of batch
    i_end = min(len(notes_data), i+batch_size)
    meta_batch = notes_data[i:i_end]
    # get ids
    ids_batch = [x['uuid'] for x in meta_batch]
    # get notes to encode
    notes = [x['note'] for x in meta_batch]
    # create embeddings (try-except added to avoid RateLimitError)
    try:
        res = openai.Embedding.create(input=notes, engine=embed_model)
    except Exception as e:
        # handle the exception by printing a message
        print(f"An exception occurred: ")#{repr(e)}")
        done = False
        while not done:
            #sleep(5)
            try:
                res = openai.Embedding.create(input=notes, engine=embed_model)
                done = True
            except Exception as e:
                print(f"Still getting an exception: {e} ... Passing")
                print(notes)
                pass
    embeds = [record['embedding'] for record in res['data']]
    # cleanup metadata
    meta_batch = [{
        'uuid': x['uuid'],
        'file': x['file'],
        'note': x['note']
    } for x in meta_batch]

    to_upsert = list(zip(ids_batch, embeds, meta_batch))
    # upsert to Pinecone
    index.upsert(vectors=to_upsert)

100%|██████████| 4/4 [00:08<00:00,  2.20s/it]


In [72]:
res = openai.Embedding.create(
    input=["how do I say hello in arabic?"],
    engine=embed_model
)


# retrieve from Pinecone
xq = res['data'][0]['embedding']

# get relevant contexts (including the questions)
res = index.query(xq, top_k=2, include_metadata=True)
#print(res)
filtered_matches = [x for x in res['matches'] if x['score'] > 0]
#print(filtered_matches)

In [73]:
limit = 8750

def retrieve(query, confidence=0.78):
    res = openai.Embedding.create(
        input=[query],
        engine=embed_model
    )
    # retrieve from Pinecone
    xq = res['data'][0]['embedding']
    # get relevant contexts
    res = index.query(xq, top_k=20, include_metadata=True)
    filtered_matches = [x for x in res['matches'] if x['score'] > confidence]
    return filtered_matches

def retrieve_context(query, confidence=0.78):
    res = openai.Embedding.create(
        input=[query],
        engine=embed_model
    )
    # retrieve from Pinecone
    xq = res['data'][0]['embedding']
    # get relevant contexts
    res = index.query(xq, top_k=10, include_metadata=True)
    filtered_matches = [x for x in res['matches'] if x['score'] > confidence]
    contexts = [
        "File: " + x['metadata']['file'] + "\nNote: " + x['metadata']['note'] for x in filtered_matches
    ] # the score is a variable that should be exposed to the user 

    if len(contexts) == 0:
        prompt = "I got nothing"

    # build our prompt with the retrieved contexts included
    prompt_start = (
        "Answer the question based on the context below.\n\n"+
        "Context:\n"
    )
    prompt_end = (
        f"\n\nQuestion: {query}\nAnswer:"
    )
    # append contexts until hitting limit
    for i in range(0, len(contexts)):
        if len("\n\n---\n\n".join(contexts[:i])) >= limit:
            prompt = (
                prompt_start +
                "\n\n---\n\n".join(contexts[:i-1]) +
                prompt_end
            )
            break
        elif i == len(contexts)-1:
            prompt = (
                prompt_start +
                "\n\n---\n\n".join(contexts) +
                prompt_end
            )
    return (prompt, filtered_matches)

In [74]:
fm = retrieve("xcs224", 0.70)

print(len(fm))
for fms in fm:
    print(fms['metadata']['note'])

20
xcs234 - reinforcement learning.md
# XCS234 Overview
```ccard
type: folder_brief_live
```

---

public-command-center.md
# Technical Notes

[[machine-learning]]

# Course-work

## Stanford

### Ai Certification

[[xcs224n - natural language processing]]
[[xcs224u - natural language understanding]]
[[xcs234 - reinforcement learning]]
[[xcs229ii - machine learning]]

### Product Management Certification

[[xprod110 - product management]]
[[xprod210 - mastering product management]]
[[xprod120 - product costing]]

# Hobbies

[[scuba diving|scuba diving]]
[[bucket-list |travel bucket list]]
[[camper van]]

Four Thousand Weeks.md
# Four Thousand Weeks

![rw-book-cover](https://m.media-amazon.com/images/I/71udc7ZQtVL._SY160.jpg)

## Metadata

Author: [[Oliver Burkeman]]
Full Title: Four Thousand Weeks
Category: #books
Date Highlighted: [[2022-01-08-Saturday]]

## Highlights
- What makes it unbearable is your mistaken belief that it can be cured. —CHARLOTTE JOKO BECK ([Location 29](https://

In [75]:
# first we retrieve relevant items from Pinecone
query_with_contexts, filtered_matches = retrieve_context("what is machine learning?", 0.7)
# then we complete the context-infused query
#ans = complete(query_with_contexts)
for x in filtered_matches:
    print(f"{x['metadata']['file']} --- {x['metadata']['header']} --- {x['score']}")


KeyError: 'header'

In [ ]:
a = "---foobar\n\nammar\nhusain"

my_list = [x for x in a.split("\n") if x != ""]
print(my_list)

if re.search(r'\w', a):
    print("The string starts is an alphanumeric character")
else:
    print("The string does not start with an alphanumeric character")

print(re.search(r'\w', a))


In [ ]:
a = 0

if a:
    print("foo")